In [1]:
# An example of semi-supervised node classification

using Flux
using Flux: onecold, onehotbatch
using Flux.Losses: logitcrossentropy
using GraphNeuralNetworks
using MLDatasets: Cora
using Statistics, Random
using CUDA
CUDA.allowscalar(false)

In [2]:
function eval_loss_accuracy(X, y, mask, model, g)
    ŷ = model(g, X)
    l = logitcrossentropy(ŷ[:, mask], y[:, mask])
    acc = mean(onecold(ŷ[:, mask]) .== onecold(y[:, mask]))
    return (loss = round(l, digits = 4), acc = round(acc * 100, digits = 2))
end

eval_loss_accuracy (generic function with 1 method)

In [9]:
# arguments for the `train` function 
Base.@kwdef mutable struct Args
    η = 1.0f-3             # learning rate
    epochs = 100          # number of epochs
    seed = 17             # set seed > 0 for reproducibility
    usecuda = true      # if true use cuda (if available)
    nhidden = 128        # dimension of hidden features
    infotime = 10      # report every `infotime` epochs
end

function train(; kws...)
    args = Args(; kws...)

    args.seed > 0 && Random.seed!(args.seed)

    if args.usecuda && CUDA.functional()
        device = gpu
        args.seed > 0 && CUDA.seed!(args.seed)
        @info "Training on GPU"
    else
        device = cpu
        @info "Training on CPU"
    end

    # LOAD DATA
    dataset = Cora()
    classes = dataset.metadata["classes"]
    g = mldataset2gnngraph(dataset) |> device
    X = g.features
    y = onehotbatch(g.targets |> cpu, classes) |> device # remove when https://github.com/FluxML/Flux.jl/pull/1959 tagged
    ytrain = y[:, g.train_mask]

    nin, nhidden, nout = size(X, 1), args.nhidden, length(classes)

    ## DEFINE MODEL
    model = GNNChain(GCNConv(nin => nhidden, relu),
                     GCNConv(nhidden => nhidden, relu),
                     Dense(nhidden, nout)) |> device

    opt = Flux.setup(Adam(args.η), model)

    display(g)

    ## LOGGING FUNCTION
    function report(epoch)
        train = eval_loss_accuracy(X, y, g.train_mask, model, g)
        test = eval_loss_accuracy(X, y, g.test_mask, model, g)
        println("Epoch: $epoch   Train: $(train)   Test: $(test)")
    end

    ## TRAINING
    report(0)
    for epoch in 1:(args.epochs)
        grad = Flux.gradient(model) do model
            ŷ = model(g, X)
            logitcrossentropy(ŷ[:, g.train_mask], ytrain)
        end

        Flux.update!(opt, model, grad[1])

        epoch % args.infotime == 0 && report(epoch)
    end
end

train()

GNNGraph:
  num_nodes: 2708
  num_edges: 10556
  ndata:
	val_mask = 2708-element Vector{Bool}
	targets = 2708-element Vector{Int64}
	test_mask = 2708-element Vector{Bool}
	features = 1433×2708 Matrix{Float32}
	train_mask = 2708-element Vector{Bool}

Epoch: 0   Train: (loss = 1.9487f0, acc = 13.57)   Test: (loss = 1.9464f0, acc = 13.6)


┌ Info: Training on CPU
└ @ Main c:\Users\HP\Desktop\VicWorks03082024\eSGCe_cora.ipynb:22


Epoch: 10   Train: (loss = 1.541f0, acc = 94.29)   Test: (loss = 1.7289f0, acc = 69.0)
Epoch: 20   Train: (loss = 0.8189f0, acc = 97.86)   Test: (loss = 1.2764f0, acc = 77.4)
Epoch: 30   Train: (loss = 0.25f0, acc = 98.57)   Test: (loss = 0.8154f0, acc = 80.8)
Epoch: 40   Train: (loss = 0.0615f0, acc = 100.0)   Test: (loss = 0.6465f0, acc = 79.4)
Epoch: 50   Train: (loss = 0.0193f0, acc = 100.0)   Test: (loss = 0.6278f0, acc = 79.1)
Epoch: 60   Train: (loss = 0.0089f0, acc = 100.0)   Test: (loss = 0.6436f0, acc = 79.2)
Epoch: 70   Train: (loss = 0.0055f0, acc = 100.0)   Test: (loss = 0.6587f0, acc = 79.3)
Epoch: 80   Train: (loss = 0.004f0, acc = 100.0)   Test: (loss = 0.6686f0, acc = 79.3)
Epoch: 90   Train: (loss = 0.0032f0, acc = 100.0)   Test: (loss = 0.6752f0, acc = 79.3)
Epoch: 100   Train: (loss = 0.0027f0, acc = 100.0)   Test: (loss = 0.6806f0, acc = 79.2)
